# low-rank Krylov Subspace Approximation (KSA) for density matrix SCF optimization

In [2]:
import sys
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")
import seqm
import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
from seqm.seqm_functions.read_xyz import read_xyz

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
device = torch.device('cpu')
seqm.seqm_functions.scf_loop.debug = True


In [3]:
%%time
DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
files = [
        'C380.xyz',
        ]

species, coordinates = read_xyz(files)
species = torch.as_tensor(species,dtype=torch.int64, device=device)[:]
coordinates = torch.tensor(coordinates, device=device, dtype=DTYPE)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

# parameters dict for KSA-XL-BOMD needs to be passed to 'scf_converger'
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [3,xl_bomd_params], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                                         # [3, xl_bomd_params], low-rank KSA scf optimization
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }

molecules = Molecule(const, seqm_parameters, coordinates, species).to(device)

### Create electronic structure driver:
esdriver = Electronic_Structure(seqm_parameters).to(device)

### Run esdriver on molecules:
esdriver(molecules)


step, DM rmse, dE, number of not converged
1 [38.89585056] [1784688.13712186] 1 1.1389927864074707
2 [43.09559091] [222.338752] 1 1.1497883796691895
3 [39.83835278] [369.5077898] 1 1.1408848762512207
4 [31.98035692] [420.72364697] 1 1.1621685028076172
5 [26.04837395] [107.55478305] 1 1.1518521308898926
6 [20.92210945] [73.82491791] 1 1.1438593864440918
7 [18.24912234] [207.68592423] 1 1.1389870643615723
8 [16.47453509] [305.90078104] 1 1.135298490524292
9 [13.55256798] [609.76548117] 1 1.1518535614013672
10 [16.85997534] [20.2590198] 1 1.129075288772583
11 [14.67958913] [38.79181879] 1 1.1499593257904053
12 [12.63424165] [24.30224028] 1 1.1400365829467773
13 [11.88848249] [74.36457573] 1 1.1602375507354736
14 [10.29609848] [95.44455183] 1 1.1575496196746826
15 [9.33808629] [76.62338079] 1 1.1451056003570557
16 [9.3832641] [22.45359966] 1 1.1390318870544434
17 [8.36396182] [69.95919592] 1 1.1452994346618652
18 [7.30729064] [25.96859532] 1 1.1566007137298584
19 [6.56085645] [65.16397814]

In [4]:
print(' Total Energy (eV):\n', molecules.Etot)
print('\n Electronic Energy (eV): ', molecules.Eelec)
print('\n Nuclear Energy (eV):\n', molecules.Enuc)
print('\n Heat of Formation (ev):\n', molecules.Hf)
print('\n HOMO-LUMO gap (eV):\n', molecules.e_gap)
print('\n Forces (eV/A):\n', molecules.force)

 Total Energy (eV):
 tensor([-48587.7154])

 Electronic Energy (eV):  tensor([-1787645.6602])

 Nuclear Energy (eV):
 tensor([1739057.9448])

 Heat of Formation (ev):
 tensor([138.2180])

 HOMO-LUMO gap (eV):
 tensor([2.7304])

 Forces (eV/A):
 tensor([[[ 8.8127e-05,  4.9092e-04, -6.4488e-04],
         [-3.7722e-04,  2.8213e-04, -4.2889e-04],
         [ 1.1399e-04,  2.4805e-05, -4.0009e-04],
         ...,
         [-2.6917e-04,  2.6461e-04, -1.3600e-04],
         [ 4.3344e-04, -2.9772e-04,  5.4880e-04],
         [-3.3322e-03,  4.8833e-04,  1.0369e-04]]])
